[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io/)

# AI Program

## Essential Matrix Calculus - Auto Diff

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 05/02/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0006AutoDiff.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
import autograd.numpy as anp
from autograd import grad
from autograd import elementwise_grad as egrad

# Miscellaneous
import os
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
from matplotlib.colors import LogNorm, Normalize, PowerNorm
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4); #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3); #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3); #<! Notation for nD array (Tensor)
cCell      = cell(3, 1); #<! Notation for a cell array
oObj       = MyClass(); #<! Notation for an object
dfData     = ps.DataFrame(); #<! Notation for a data frame
hObj       = plt.Axes(); #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions

In [ ]:
# Parameters

## Auto Differentiation

The concept of _Auto differentiation_ is being able to calculate the gradient or higher order derivatives of a function without explicit procedure.  
The vision is being able to 

This notebook uses one of the pioneer packages in the field, [`AutoGrad`](https://github.com/HIPS/autograd).   
This package have inspired `PyTorch` and `JaX` which are _State of the Art_ auto differentiation packages.

`AutoGrad` works by overloading a sub set of _NumPy_ and _SciPy_ functions.  
In order to use it, a function must replace the operations of `numpy` by `autograd.numpy`.

* <font color='brown'>(**#**)</font> This notebook doesn't cover the ideas for _auto differentiation_. One might watch [What is Automatic Differentiation](https://www.youtube.com/watch?v=wG_nF1awSSY).
* <font color='brown'>(**#**)</font> There are few approaches to auto differentiation. One of them is based on Dual Numbers.  
  See A Hands On Introduction to Automatic Differentiation: [Part I](https://mostafa-samir.github.io/auto-diff-pt1), [Part II](https://mostafa-samir.github.io/auto-diff-pt2).

In [ ]:
# Parameters

# Data
numCoeff    = 3
numSamples  = 50
noiseStd    = 0.15

# Model
λ  = 0.1 #<! Regularization
mD = -np.eye(numSamples - 1, numSamples, k = 0) + np.eye(numSamples - 1, numSamples, k = 1) #<! Finite Differences Matrix
δ  = 1 #<! Huber Loss

# Visualization
numGrdiPts = 1000


In [ ]:
# Auxiliary Functions



## Exercise 001

1. Compute the directional derivative $\nabla f \left( \boldsymbol{x} \right) \left[ \boldsymbol{h} \right]$ and the gradient $\nabla f \left( \boldsymbol{x} \right)$ of:

$$ f \left( \boldsymbol{x} \right) = \boldsymbol{x}^{T} \boldsymbol{A} \boldsymbol{x} $$

2. Implement the gradient function a Python function.
3. Implement the function using `AutoGrad`.


## Solution 001

Since we can write $f \left( \boldsymbol{x} \right) = \left \langle \boldsymbol{x}, A \boldsymbol{x} \right \rangle$:

$$
\begin{aligned}
\nabla f \left( \boldsymbol{x} \right) \left[ \boldsymbol{h} \right] & = \left \langle \boldsymbol{h}, A \boldsymbol{x} \right \rangle + \left \langle \boldsymbol{x}, A \boldsymbol{h} \right \rangle && \text{Product rule and linearity} \\
& = \left \langle A \boldsymbol{x}, \boldsymbol{h} \right \rangle + \left \langle \boldsymbol{x}, A \boldsymbol{h} \right \rangle && \text{Symmetry} \\
& = \left \langle A \boldsymbol{x}, \boldsymbol{h} \right \rangle + \left \langle {A}^{T} \boldsymbol{x}, \boldsymbol{h} \right \rangle && \text{Adjoint (Or just equality)} \\
& = \left \langle \left( A + {A}^{T} \right) \boldsymbol{x}, \boldsymbol{h} \right \rangle && \text{Linearity} \\
& \Rightarrow \nabla f \left( \boldsymbol{x} \right) = \left( A + {A}^{T} \right) \boldsymbol{x}
&& \blacksquare
\end{aligned}
$$

In [ ]:
# The Gradient Function

#===========================Fill This===========================#
# 1. Implement the gradient function.
# !! You may find the `@` operator useful.

def GradF( mA: np.ndarray, vX: np.ndarray ) -> np.ndarray:

    return (mA + mA.T) @ vX

#===============================================================#

In [ ]:
# The Function

#===========================Fill This===========================#
# 1. Implement the function.
# !! Use `anp` instead of `np`.

def FunctionF( mA: np.ndarray, vX: np.ndarray ) -> np.ndarray:

    return anp.dot(anp.dot(vX, mA), vX)

#===============================================================#

In [ ]:
# Data
mA = anp.random.rand(5, 5)
vX = anp.random.rand(5)

# Defining the functions (Single argument)
hF          = lambda vX: FunctionF(mA, vX)
hGradF      = lambda vX: GradF(mA, vX)
hAutoGradF  = grad(hF) #<! AutoGrad

print(f'Implementation of the analytic gradient is verified: {np.allclose(hGradF(vX), hAutoGradF(vX))}')